# Dissociating task-engagement and arousal in primary auditory cortex
**Charlie Heller and Stephen David**

# Introduction
It is well-known in the field of auditory neuroscience that the evoked spiking responses of neurons in auditory cortex are variable from trial to trial. Put another way, the same neuron will respond with different numbers of spikes to repeated presentations of identical sensory stimuli. Recent work has shown that some of this variability can be explained by task engagement. Often sound-evoked responses increase during task-engagement. This increase is thought to play a role in enhancing sensory encoding by increasing the signal to noise ratio of neurons that encode sounds relevant to the task. However, it has also been noted that other state variables, such as global arousal, can also modulate the strength of resonses in the same areas of auditory cortex auditory cortex. The interaction of these two state variables on auditory encoding is still an open question.

In order to study this, our lab records neural activity from several (10-30) neurons simulataneously while both manipulating the animal's behavioral state using a tone-detection task and while monitoring the animals arousal using continuous measurements of pupil size. 

<img src="data/FIgure06_ptd_behavior.png" alt="drawing" width="500"/>

Today, we will use Python to learn about one analytical method to dissciate the effects of arousal from task-engagement. We will use the data from one neuron recorded in the primary auditory cortex of an awake, behaving ferret to do this.

Here are PSTH responses from a neuron that appears to be modulated by both task engagement and arousal:

<img src="data/psth_examples.png" alt="drawing" width="600"/>

**Problem:** Arousal state (indexed by pupil) correlates with task engagement. 

**Question:** Do both state variables actually produce changes in this neuron's sound-evoked activity?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import os.path

# Load the data
The data has been stored in `numpy` arrays. You've worked with Numpy arrays in the past (e.g., when analyzing patch-clamp data). We have saved a vector of spike counts for one A1 neuron, along with the animal's pupil size, and the behavioral state (engaged or passive listening). Let's start by loading the data.

In [ ]:
data_path = 'data' # path relative to notebook location
spike_counts = np.load(os.path.join(data_path, 'TAR010c-06-1.npy'))
pupil_size = np.load(os.path.join(data_path, 'pupil.npy'))
behavior_state = np.load(os.path.join(data_path, 'behavior.npy'))

Let's take a look at our three arrays (`spike_counts`, `pupil_size`, and `behavior_state`):

In [ ]:
# the shape of our numpy arrays
print("spike_counts shape: {}".format(spike_counts.shape))
print("pupil_size shape: {}".format(pupil_size.shape))
print("behavior_state shape: {}".format(behavior_state.shape))

In this experiment we have collected 359 trials that are 1.45 seconds long. Each trial contains one distractor sound (0.35-1.10 sec). Trials with target sounds have been removed for simplicity. 

We have grouped the data into 20 time bins per second (i.e., our sampling rate is 20 Hz). Twenty bins per second times 1.45 seconds gives us a total of 29 bins. This gives us the dimensions of our three arrays:

* the first dimension corresponds to the trials
* the second dimension corresponds to the time bins

Let's visualize this to get a better sense of the data we're working with.

In [ ]:
# plt.subplots allows you to easily control the layout of the subplots in your figure

# create the figure and the axes objects
f, ax = plt.subplots(1, 3, figsize=(12, 5), sharex=True, sharey=True)

# plot the spike counts on the first axis
img = ax[0].imshow(spike_counts, aspect='auto', origin='lower', extent=(0, 1.45, 1, 359))
plt.colorbar(img, ax=ax[0])
ax[0].set_xlabel('Time (sec)')
ax[0].set_ylabel('Trial number')
ax[0].set_title('Number of spikes per bin')

# plot the pupil size on the first axis
img = ax[1].imshow(pupil_size, aspect='auto', origin='lower', extent=(0, 1.45, 1, 359))
plt.colorbar(img, ax=ax[1])
ax[1].set_xlabel('Time (sec)')
ax[1].set_title('Pupil diameter (mm)')

# plot the behavior state on the first axis
img = ax[2].imshow(behavior_state.astype('i'), aspect='auto', origin='lower', extent=(0, 1.45, 1, 359))
ax[2].set_xlabel('Time (sec)')
ax[2].set_title('Behavior\nyellow = active, purple = passive')
plt.colorbar(img, ax=ax[2])

f.tight_layout()

On the y axis we've plotted stimulus repetitions, on the x axis, we've plotted time bins. For both `spike_counts` and `pupil_size`, the values of the data change continuously over the trial (albeit slowly for pupil). We can even tell from looking at `spike_counts` that there is a time window in which firing rate increases. This is between sound onset and offset. For `behavior_state`, however, we see that the values are consistent over the duration of individual stimulus repetitions (they're either 0 or 1). This is because the behavioral state of the animal (whether or not they're performing a task) never changes during a trial. So, `behavioral_state` is just a binary mask telling us when the animal was behaving or passively listening. In contrast, `spike_counts` and `pupil_size` are continuous variables. 

# Plot state-dependent PSTHs

One observation based on the plots above is that the number of spikes evoked by a sound seems to correlate with both the animal's pupil size and its behavioral state. Let's take a closer look at this by plotting the PSTH (mean stimulus response base don the spike count) for the following conditions:

* trials when the mean pupil is greater than its median
* trials when the mean pupil is less than its median
* trials when the animal is behaving (`behavioral_state == True`)
* trials when the animal is passively listening (`behavioral_state == False`)

We'll help you do this for pupil, then you can try for active/passive listening.

First, we need to create a `pupil_mask` to classify each trial as large or small pupil. This will be a mask, in the same way as `behavior_state`. To do this, we're going to work with the mean pupil size on each trial.

## Exercise
Create a new array, called `mean_pupil_per_trial` that contains the mean value of the pupil on each trial. 
* Hint: This should have the same length as the number of trials.

In [ ]:
# Answer
mean_pupil_per_trial = np.mean(pupil_size, axis=1)
mean_pupil_per_trial.shape

Now that we have this, let's create a `pupil_mask` (1D array of True/False values), based on if the mean value of pupil was above or below the median value.

In [ ]:
# create variable to hold median pupil size
pupil_divider = np.median(pupil_size)

# create a pupil mask (like behavior_state)
pupil_mask = mean_pupil_per_trial >= pupil_divider

## Exercise
Finally, plot an image of the mask alongside pupil to make sure this seems to have worked as we hoped, using the same image format as above. Use plotting code from above to figure out how to display both `pupil_mask` and `pupil_size` in subplots on the same graph. In this case, you'll just have two subplots: `ax[0]` and `ax[1]`.
* Hint: your mask is one dimensional. If you want to display it as an image (like above) we need to add an axis to make it two dimensional. To do this, use `np.expand_dims`. Google this if you get stuck. Be sure to save the 2D array as a new variable, e.g., `pupil_mask_2D`.

In [ ]:
# Answer

# expand the pupil mask
pupil_mask_2D = np.expand_dims(pupil_mask, axis=1)

# visualize the mask
f, ax = plt.subplots(1, 2, sharex=True, sharey=True)

ax[0].imshow(pupil_size, aspect='auto', origin='lower', extent=(0, 1.45, 1, 359))
ax[0].set_ylabel('Stimulus repetitions')
ax[0].set_xlabel('Time bins')

ax[1].imshow(pupil_mask_2D, aspect='auto', origin='lower', extent=(0, 1.45, 1, 359))
ax[1].set_xlabel('Time bins')

f.tight_layout()

You should see yellow bands aligned with trials where pupil is large (if you don't, ask one of the TAs for help). Now, let's use the mask to extract only the spike data when pupil is big and save this as a new array called `big_pupil_spikes`. Do the same for small pupil and save this as an array called `small_pupil_spikes`. Recall that the length of our mask is equal to the number of trials and that our data has shape: `(nTrials, nBins)`.

In [ ]:
big_pupil_spikes = spike_counts[pupil_mask, :]

# The "~" operator inverts boolean values
small_pupil_spikes = spike_counts[~pupil_mask, :]

Note that in order to mask our data, we masked only the single dimension of our `spike_count` array corresponsing to trials because we know `pupil_mask` is constant for the duration of every trial. 

Finally, we can compute the PSTH's for each of these conditions and plot them:

## Exercise
Using `np.mean()` along the appropriate axis, compute the big and small pupil PSTH's. Save them as `big_pupil_psth` and `small_pupil_psth`. 
* Hint: the final dimensions of both should be `(29,)`.

In [ ]:
# Answer
big_pupil_psth = np.mean(big_pupil_spikes, axis=0)
small_pupil_psth = np.mean(small_pupil_spikes, axis=0)

Once you've got the PSTH's calculated correctly, run the code below to plot them.

In [ ]:
# plot them
f, ax = plt.subplots(1,1)
ax.plot(big_pupil_psth, color='purple')
ax.plot(small_pupil_psth, color='orchid')
ax.legend(['Big pupil', 'Small pupil'])
ax.set_xlabel('Time bins')
ax.set_ylabel('Spike counts');

Cool, as we noted in our original plot of trial by trial data, the spiking response to a sound by this cell seems to depend strongly on pupil size.

## Exercise
Repeat the above procedure using the `behavioral_state` mask. There will be a couple of steps required to do this:
* First step: mask the data (remember that behavior state is constant over the duration of a trial, so you shouldn't need to use the entire `(359, 29)` mask for this, just the first column)
    * Save the masked data as `active_spikes` and `passive_spikes`
* Second step: compute the PSTH using `np.mean()`
    * Save the PSTHs as `active_psth` and `passive_psth`

In [ ]:
# Answer

# mask the data
active_spikes = spike_counts[behavior_state[:,0], :]
passive_spikes = spike_counts[~behavior_state[:,0], :]

# compute the psth
active_psth = np.mean(active_spikes, axis=0)
passive_psth = np.mean(passive_spikes, axis=0)

Now that we've got the behavior PSTH's calculated, let's plot the results alongside the pupil PSTH's.

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(10, 4), sharex=True, sharey=True)

ax[0].plot(big_pupil_psth, color='purple')
ax[0].plot(small_pupil_psth, color='orchid')
ax[0].legend(['Big pupil', 'Small pupil'])
ax[0].set_xlabel('Time bins')
ax[0].set_ylabel('Spike counts')

ax[1].plot(active_psth, color='red')
ax[1].plot(passive_psth, color='lightcoral')
ax[1].legend(['Active', 'Passive'])
ax[1].set_xlabel('Time bins')

f.tight_layout()

Just like for pupil, the spiking response also apparently depends strongly on whether or not the animal is engaged in a task. However, we saw in the beginning that both pupil and behavior state are highly correlated. How then do we dissociate which of these two variables is truly responsible for the change in firing rates we observed?

# Stepwise Linear Regression

To disscoiate the effects of pupil and behavior state in this cell, we can perform what is called a stepwise linear regression. Essentially, this method will tell us how much variance in spike counts can be uniquely attributed to pupil alone and to behavior alone. Before we get into this, let's start by only considering pupil and performing a simple linear regression.

As a refresher on linear regression, consider the cartoon example below. Ignore the code, this is just meant to be illustrative.

In [ ]:
X1 = np.random.random(100)*100-50
X2 = np.random.random(100)*100-50
Y = 100*(np.random.random(100)-0.5) + (3*X1) + 50

f, ax = plt.subplots(1, 2, figsize=(8,4))
ax[0].set_title('Strong linear relationship')
ax[0].plot(X1, Y, 'k.')
ax[0].plot(X1, 3*X1 + 50, 'r-')
ax[0].set_xlabel('Independent variable #1 - x1')
ax[0].set_ylabel('Dependent variable - y')

ax[1].set_title('No linear relationship')
ax[1].plot(X2, Y, 'k.')
ax[1].axhline(50, color='red', linestyle='-')
ax[1].set_xlabel('Independent variable #2 - x2')

f.tight_layout()

In the cartoon above, we've simulated an experiment with two *independent variables*, $x_1$ and $x_2$ (e.g., pupil and engagement), and a single *dependent variable*, $y$, whose value may depend on one or both of the independent variables. 

On the left, we've plotted $x_1$ against $y$ (black dots), where there is a strong positive linear relationship between them. This relationship is well modeled by a line (red),
$$ y = m x_1 + b $$
$m$ and $b$ are the parameters fit by the linear regression model, which for this plot are $m=3$ and $b=50$.

On the right, the scatter plot looks random, and $m=0$ and $b=50$.

Our first goal is to fit a linear regression model that uses a single indepdent variable, pupil size, to predict a dependent variable, spike counts. In "regression-ese", the model looks like this:
$$ \hat{r}(t) = \beta_{1}p(t) + \beta_{0} $$
$\hat{r}$ is the predicted spike count, $p(t)$ is pupil size, and $\beta_{0}$ and $\beta_{1}$ are model parameters. "Beta", $\beta$, is the standard variable name for regression parameters. Compare this with the formula above and you'll see that $\beta_{1}$ maps to $m$, $\beta_{0}$ maps to $b$ and $p(t)$ maps to $x_{1}$. They are the same formula, but expressed using different notation.

## Exercise
To perform regression, we must first "unwrap" our data into a one-dimensional, continuous trace over time. In other words, we must `reshape` it from shape: `(n_repetitions, n_time_bins)` into `(n_repetitions * n_time_bins)`. Using `np.reshape`, do this for `spike_counts` and `pupil_size`. Save the results into `spike_counts_long` and `pupil_size_long`, respectively.
* Hint: Is there a trick you can use to "flatten" the array

In [ ]:
# Answer
spike_counts_long = np.reshape(spike_counts, -1)
pupil_size_long = np.reshape(pupil_size, -1)

# or...
spike_counts_long = np.reshape(spike_counts, spike_counts.shape[0]*spike_counts.shape[1])
pupil_size_long = np.reshape(pupil_size, spike_counts.shape[0]*spike_counts.shape[1])

# or...
spike_counts_long = spike_counts.flatten()
pupil_size_long = pupil_size.flatten()

# or ..
spike_counts_long = spike_counts.ravel()
pupil_size_long = pupil_size.ravel()

Now that we have one long time vector of data, let's visualize what this looks like.

In [ ]:
f, ax = plt.subplots(2, 1, figsize=(10, 4), sharex=True)

ax[0].plot(spike_counts_long, color='grey')
ax[0].set_ylabel('Spike counts')

ax[1].plot(pupil_size_long, color='purple')
ax[1].set_ylabel('pupil size')
ax[1].set_xlabel('Time bins')

f.tight_layout()

Cool, with this view, we can clearly see that the spike count over time seems to correlate with pupil. Now, can we use pupil to predict these changes over time? To answer this question, we'll use linear regression. Linear regression is the process of finding the best fit line to the data. This is perhaps better understood when visualized as below (modeled after the above cartoons): 

In [ ]:
f, ax = plt.subplots(1,1)

ax.plot(pupil_size_long, spike_counts_long, 'k.')
ax.set_xlabel('Pupil size')
ax.set_ylabel('Spike count');

Though the relationship isn't quite as strong as the cartoon above, we see that as expected, there is positive relationship between pupil size and spike count. Now, using linear regression, we can ask how much of the variance in spike counts can be explained using pupil size. To do this, we'll use the Ordinary Least Squares (`OLS`) model from the `statsmodels` package. This requires a few steps:
* Create your input (independent) and output (dependent) variables. This is pupil size and spike count for us.
    * Note that in order to find both a slope and intercept, you must explicitly add a constant to the independent variable!
    * Note also, we format the input and output variables as dataframes so that the model output contains variable names that make sense
* Create the model
* Fit the model
* Look at the results

In [ ]:
# set our dependent variable
Y = pd.DataFrame(data=spike_counts_long, columns=['spikeCounts'])

# set our independent variable
X = pd.DataFrame(data=pupil_size_long, columns=['pupilSize'])

# fit both a slope AND intercept
X = sm.add_constant(X)

# let's pause and look at our model inputs
print(X.head())
print(Y.head())

The above is pretty straightforward. However, note that all `sm.add_constant()` does, is add a column to the input of all 1's. Thus, the resulting model parameter for `const` will be just that, a constant, over all time. In our example, you can think of this as the spike count of the cell if pupil size were to be 0.

In [ ]:
# instantiate the model
model = sm.OLS(Y, X)

# fit the model
results = model.fit()

# look at a summary of results
results.summary()

There is a lot of information that is output here. For our purposes, let's focus just on the variance explained by the model, `R^2`, and the model fit parameters: `const` (i.e., $\beta_{0}$) and `pupilSize` (i.e., $\beta_{1}$).

Thus, plugging in the model fit parameters, we arrive at the following equation describing the relationship between pupil and spike counts in this cell:

$$\hat{r}(t) = 0.013 p(t) + 0.029$$

where $\hat{r}(t)$ is the predicted spike count and $p(t)$ is the pupil size.

Based on the `R^2` value we see that this model does a very poor job of capturing the variability in spike counts. We would interpret this value of 0.038 as saying that pupil accounts for only 3.8 percent of the variability in spike counts. This is, perhaps, unsurprising. There is not information about the sound evoked response in this model, which is the major source of variability in the spike counts. Before we move on to a slightly more sophisticated model, let's plot the predicted response based on this model.

## Exercise
Search for a method of the model results called `predict` and use this to create an array of spike count predictions for our model input vector `X`. Save this as `spike_counts_pred`.
* Why do you think we have to use `X` instead of `pupil_size_long`? Hint, what's the shape of `X`? The shape of `pupil_size_long`?

In [ ]:
# Answer 
spike_counts_pred = results.predict(X)

Now that we've got the prediction, let's plot it over the true response.

In [ ]:
plt.figure(figsize=(13,4))

ax1 = plt.subplot2grid((2, 3), (0,0), colspan=2)
ax2 = plt.subplot2grid((2, 3), (1,0), colspan=2, sharex=ax1)
ax3 = plt.subplot2grid((2,3), (0,2), rowspan=2)

ax1.plot(spike_counts_long, color='gray')
ax1.plot(spike_counts_pred, color='k')
ax1.legend(['response', 'prediction'])
ax1.set_ylabel('Spike counts')

ax2.plot(pupil_size_long, color='purple')
ax2.set_ylabel('pupil size')
ax2.set_xlabel('Time bins')

ax3.plot(pupil_size_long, spike_counts_long, 'k.')
ax3.plot(pupil_size_long, spike_counts_pred, 'r-')
ax3.legend(['raw data', 'prediction'])
ax3.set_ylabel('Spike counts')
ax3.set_xlabel('Pupil size')
plt.tight_layout()

Now, as we mentioned, the above model doesn't perform all that well. This is mainly because it doesn't take into account the stimulus evoked activity in the spike count. This is evident when you compare the gray to black trace in the above plot. The prediction (black) is smooth like pupil, while the response (gray) is much more dynamic. What we really want is a model that uses pupil to **scale** the **mean** stimulus evoked response of the neuron (i.e., the PSTH we generated earlier). In other words, we know that sound drives this cell. We would like to understand how pupil modulates the sound evoked response. Therefore, in addition to using just pupil, we'll also include a term for pupil **times** the mean response of the cell. Thus our model will look like this:

$$\hat{r}(t) = \beta_{3} r_{0}(t) p(t) + \beta_{2} r_{0}(t) + \beta_{1} p(t) + \beta_{0}$$

where $\hat{r}(t)$ is the predicted spike count, $r_{0}(t)$ is the mean spike count on each trial (PSTH), and $\beta_{0}: \beta_{3}$ are the parameters that will be fit by the model. 

This is starting to look a little complicated. We now have what is called a multiple regression model. We can also write the equation as follows:

$$\hat{r}(t) = (\beta_{1} p(t) + \beta_{0}) + (\beta_{3} p(t)  + \beta_{2}) r_{0}(t)$$

Where we can now think of the entire first term as our old model (pupil only) and the second term as our new addition to incorporate the stimulus response. So really, it's like we've combined two different linear regression models into one. in the first model, our independent variable is simply $p(t)$ and in our second model the indendent variable is $r_{0}(t)p(t)$. 

Now, in order to implement this model, there are a couple of additional steps:
* Compute the PSTH
* "tile" the PSTH over all time so that it is the same length as `pupil_size_long`
* Create our new `X` array
* Fit the model as above

## Exercise
Compute the psth over all stimuli of the the spike counts. Remember that our raw spike counts are saved in an array called: `spike_counts`. Save this as `psth`

In [ ]:
# Answer
psth = np.mean(spike_counts, axis=0)

## Exercise
`tile` (hint) this psth over time to make it the same length as `pupil_size_long`. Call this new variable `psth_long`.
* Hint - the argument to `np.tile` called `reps` must be an `int`

In [ ]:
# Answer
ntile = int(pupil_size_long.shape[0]/spike_counts.shape[-1])
psth_long = np.tile(psth, ntile)

Now that we've generated `psth_long`, let's illustrate what it looks like.

In [ ]:
f, ax = plt.subplots(2, 1, figsize=(10, 4), sharex=True)

#ax[0].plot(spike_counts_long, color='grey')
ax[0].plot(psth_long, color='black')
ax[0].set_ylabel('Spike counts')

ax[1].plot(pupil_size_long, color='purple')
ax[1].set_ylabel('pupil size')
ax[1].set_xlabel('Time bins')

f.tight_layout()

That's a bit dense. Let's zoom in to see the repetitions of the individual PSTHs.

In [ ]:
ax[0].axis(xmin=0, xmax=500)
display(f)

Now that we've got our $r_{0}(t)$ (`psth_long`). Let's create our new `X` DataFrame:

In [ ]:
X = pd.DataFrame({
    'p(t)*r0(t)': pupil_size_long * psth_long,
    'r0(t)': psth_long,
    'p(t)': pupil_size_long,
})
X = sm.add_constant(X)
X.head()

Now our independent variable has 4 columns, one for each of the model parameters $\beta_{0}:\beta{3}$ that we are trying to find.

## Exercise
Create the model, fit the model, and look at a summary of our results.

In [ ]:
# Answer
model = sm.OLS(Y, X)

results = model.fit()

results.summary()

This model accounts for 28.4 percent of the variance in spike counts. Thus, it performs significantly better than the first model. 

Let's look at the prediction as we did before.

In [ ]:
# create prediction
pred = results.predict(X).values

f, ax = plt.subplots(2, 1, figsize=(10,4), sharex=True)

ax[0].plot(spike_counts_long, color='gray')
ax[0].plot(pred, color='k')
ax[0].legend(['response', 'prediction'])
ax[0].set_ylabel('Spike counts')

ax[1].plot(pupil_size_long, color='purple')
ax[1].set_ylabel('pupil size')
ax[1].set_xlabel('Time bins')

plt.tight_layout()

Nice. It's clear from this view that this second model does indeed capture the activity of this cell better than the first model. One last way we can visualize this is by plotting the pupil-dependent PSTH of the model fit on top of the true pupil-dependent PSTH to see if the model captures the changes we observed before. To to this, let's first compute the pupil-dependent PSTH of the prediction. This will require `reshape`ing our prediction, masking it, then computing the mean. For the sake of time, we've done this for you.

In [ ]:
# reshape the data
pred_folded = pred.reshape(spike_counts.shape[0], spike_counts.shape[1])

# mask the data
big_pupil_pred_spikes = pred_folded[pupil_mask, :]
small_pupil_pred_spikes = pred_folded[~pupil_mask, :]

# compute the psth
big_pupil_pred_psth = np.mean(big_pupil_pred_spikes, axis=0)
small_pupil_pred_psth = np.mean(small_pupil_pred_spikes, axis=0)

Now we can plot this:

In [ ]:
# plot them
f, ax = plt.subplots(1,1)
ax.plot(big_pupil_psth, color='purple')
ax.plot(small_pupil_psth, color='orchid')
ax.plot(big_pupil_pred_psth, '--', color='purple')
ax.plot(small_pupil_pred_psth, '--', color='orchid')
ax.legend(['Big pupil', 'Small pupil', 'Big pupil pred', 'Small pupil pred'])
ax.set_xlabel('Time bins')
ax.set_ylabel('Spike counts')

Cool! Our predictions overlap nicely with the true pupil-dependent PSTH's. However, as we noted earlier, our goal is to determine if this effect is due to pupil or to task-engagement. To dissociate these effects, we will create yet another set of linear regression models, this time with `behavior_state` **and** `pupil_size` as predictors. 

We will then remove each predictor one by one (in a stepwise manner) and analyse how `R^2` changes as a result. The way this is done is by **shuffling** (randomly reordering) each predictor in time. If there is any relationship between the predictor and the spike count, model performance should decrease for the shuffled model relative to the non-shuffled model. Shuffling keeps the degrees of freedom in the model the same and thus allows for simpler comparison of performance with the original (unshuffled) model. 

In short, this means we will need to create four models to answer our question:
* Pupil not shuffled and Behavior not shuffled model (full model)
* Pupil shuffled and Behavior not shuffled model (behavior only model)
* Pupil not shuffled and Behavior shuffled model (pupil only model)
* Pupil shuffled and Behavior shuffled model (shuffled model)

Where the full model looks like this:

$$\hat{r}(t) = [ \beta_{3} b(t) + \beta_{1} p(t) + \beta_{0} ] + 
 [ \beta_{6} b(t)  + \beta_{5} p(t)  + \beta_{4} ] r_{0}(t) $$

Again, this looks complicated, but all we've done is replicate the first two terms from the model above and replaced $p(t)$ with $b(t)$ for behavioral state. That makes six free regression parameters, but the model fitting procedure is the same as before.

We'll start this off for you by creating the full model. But first, we need to unwrap `behavior_state` into a long vector, like `pupil_size_long`.

In [ ]:
behavior_state_long = behavior_state.ravel()

So now our situation looks something like this where we have two state variables (pupil and behavior) that we can use to predict the spike count in a given time bin.

In [ ]:
f, ax = plt.subplots(3, 1, sharex=True)

ax[0].plot(spike_counts_long, color='grey')
ax[0].set_ylabel('Spike counts')

ax[1].plot(pupil_size_long, color='purple')
ax[1].set_ylabel('pupil size')

ax[2].plot(behavior_state_long, color='red')
ax[2].set_ylabel('behavior state')
ax[2].set_xlabel('Time bins')

f.set_figwidth(10)

plt.tight_layout()

Let's first create the variables we'll need to build our models (i.e. we need to combine pupil with the psth and behavior with the psth to capture the evoked auditory responses of the cell):

In [ ]:
# create dependent variables
behavior_signal = psth_long * behavior_state_long
pupil_signal = psth_long * pupil_size_long

Now we can build our models.

**Full-model:**

In [ ]:
X_full = pd.DataFrame({
    'b(t)*r0(t)': behavior_state_long * psth_long,
    'p(t)*r0(t)': pupil_size_long * psth_long,
    'b(t)': behavior_state_long.astype('f'),
    'r0(t)': psth_long,
    'p(t)': pupil_size_long,
})
X_full = sm.add_constant(X_full)
X_full.head()

In [ ]:
Y = spike_counts_long

full_model = sm.OLS(Y, X_full)
results_full = full_model.fit()

**Behavior model (pupil shuffled):**

In [ ]:
# shuffle the pupil
pupil_shuffled = pupil_size_long.copy()
np.random.shuffle(pupil_shuffled) # shuffles in place

# create the model as before
X_behavior = pd.DataFrame({
    'b(t)*r0(t)': behavior_state_long * psth_long,
    'p(t)*r0(t)': pupil_shuffled * psth_long,
    'b(t)': behavior_state_long.astype('f'),
    'r0(t)': psth_long,
    'p(t)': pupil_shuffled,
})
X_behavior = sm.add_constant(X_behavior)

behavior_model = sm.OLS(Y, X_behavior)

results_behavior = behavior_model.fit()

## Exercise
Following the previous two examples, create the pupil model by shuffling behavior and create the shuffled model by shuffling both pupil and behavior.

**Pupil model (behavior shuffled):**

In [ ]:
# Answer
# shuffle behavior
behavior_shuffled = behavior_state_long.copy()
np.random.shuffle(behavior_shuffled) # shuffles in place
behavior_signal_shuffled = psth_long * behavior_shuffled

# create the model as before
X_pupil = pd.DataFrame({
    'b(t)*r0(t)': behavior_shuffled * psth_long,
    'p(t)*r0(t)': pupil_size_long * psth_long,
    'b(t)': behavior_shuffled.astype('f'),
    'r0(t)': psth_long,
    'p(t)': pupil_size_long,
})
X_pupil = sm.add_constant(X_pupil)

pupil_model = sm.OLS(Y, X_pupil)

results_pupil = pupil_model.fit()

**Shuffled model (both shuffled)**

In [ ]:
# Answer
X_shuffled = pd.DataFrame({
    'b(t)*r0(t)': behavior_shuffled * psth_long,
    'p(t)*r0(t)': pupil_shuffled * psth_long,
    'b(t)': behavior_shuffled.astype('f'),
    'r0(t)': psth_long,
    'p(t)': pupil_shuffled,
})
X_shuffled = sm.add_constant(X_shuffled)

shuffled_model = sm.OLS(Y, X_shuffled)

results_shuffled = shuffled_model.fit()

Now, let's take a look at the `R^2` values for the different models. (Note that we can access these as attributes of `results_` like so: `results_behavior.rsquared`. This is a nice alternative to printing our the entire summary table as we did above (though that contains useful information too).

In [ ]:
f, ax = plt.subplots(1,1)
ax.plot([0, 1, 2, 3], [results_full.rsquared, results_behavior.rsquared, results_pupil.rsquared,
                      results_shuffled.rsquared], 'o-', color='k')
ax.axhline(results_shuffled.rsquared, color='k', linestyle='--', alpha=0.5)
ax.axhline(results_full.rsquared, color='k', linestyle='--', alpha=0.5)
ax.set_xticks([0, 1, 2, 3])
ax.set_xticklabels(['full', 'behavior', 'pupil', 'shuffled'])
ax.set_ylabel('R^2')
ax.set_xlabel('Model')

From this, we see that the pupil alone accounts for the same amount of variance as the full model. The behavior model alone does better than the psth alone (shuffled model), but not as well as pupil or the shuffled model. Because there is no improvement from pupil alone to full model, we would classify this cell as an entirely pupil dependent cell. The only reason behavior does better than the psth alone is because behavior is correlated with pupil. Let's visualize this in another way using our PSTH plots. To do this, let's first create the state-dependent PSTH predictions from the pupil only model and behavior only model.

In [ ]:
# get predictions
behavior_pred = results_behavior.predict(X_behavior).values
pupil_pred = results_pupil.predict(X_pupil).values

# reshape predictions
behavior_pred = behavior_pred.reshape(spike_counts.shape[0], spike_counts.shape[1])
pupil_pred = pupil_pred.reshape(spike_counts.shape[0], spike_counts.shape[1])

# compute psth of predictions

# for behavior only model
active_behavior_pred_psth = np.mean(behavior_pred[behavior_state[:, 0], :], axis=0)
passive_behavior_pred_psth = np.mean(behavior_pred[~behavior_state[:, 0], :], axis=0)

big_pupil_behavior_pred_psth = np.mean(behavior_pred[pupil_mask, :], axis=0)
small_pupil_behavior_pred_psth = np.mean(behavior_pred[~pupil_mask, :], axis=0)

# for pupil only model
active_pupil_pred_psth = np.mean(pupil_pred[behavior_state[:, 0], :], axis=0)
passive_pupil_pred_psth = np.mean(pupil_pred[~behavior_state[:, 0], :], axis=0)

big_pupil_pupil_pred_psth = np.mean(pupil_pred[pupil_mask, :], axis=0)
small_pupil_pupil_pred_psth = np.mean(pupil_pred[~pupil_mask, :], axis=0)

In [ ]:
f, ax = plt.subplots(2,2)

ax[0,0].plot(big_pupil_psth, color='purple')
ax[0,0].plot(small_pupil_psth, color='orchid')
ax[0,0].plot(big_pupil_pupil_pred_psth, '--', color='purple')
ax[0,0].plot(small_pupil_pupil_pred_psth, '--', color='orchid')
ax[0,0].legend(['Big pupil', 'Small pupil', 'Big pred', 'Small pred'])
ax[0,0].set_ylabel('Spike counts')
ax[0,0].set_title('Pupil model')

ax[0,1].plot(active_psth, color='red')
ax[0,1].plot(passive_psth, color='lightcoral')
ax[0,1].plot(active_pupil_pred_psth, '--', color='red')
ax[0,1].plot(passive_pupil_pred_psth, '--', color='lightcoral')
ax[0,1].legend(['Active', 'Passive', 'Active pred', 'Passive pred'])
ax[0,1].set_title('Pupil model')

ax[1,0].plot(big_pupil_psth, color='purple')
ax[1,0].plot(small_pupil_psth, color='orchid')
ax[1,0].plot(big_pupil_behavior_pred_psth, '--', color='purple')
ax[1,0].plot(small_pupil_behavior_pred_psth, '--', color='orchid')
ax[1,0].legend(['Big pupil', 'Small pupil', 'Big pred', 'Small pred'])
ax[1,0].set_xlabel('Time bins')
ax[1,0].set_ylabel('Spike counts')
ax[1,0].set_title('Behavior model')

ax[1,1].plot(active_psth, color='red')
ax[1,1].plot(passive_psth, color='lightcoral')
ax[1,1].plot(active_behavior_pred_psth, '--', color='red')
ax[1,1].plot(passive_behavior_pred_psth, '--', color='lightcoral')
ax[1,1].legend(['Active', 'Passive', 'Active pred', 'Passive pred'])
ax[1,1].set_xlabel('Time bins')
ax[1,1].set_title('Behavior model')


f.set_figwidth(10)
f.set_figheight(7)
plt.tight_layout()

As expected from our `R^2` output, the Behavior model alone can't capture the pupil dependent changes in the PSTH (lower left panel). However, the pupil model alone **can** capture the behavior changes in the PSTH (upper right). This is just another way of illustrating that the spike count changes in this cell are truly just due to pupil, not task-engagement.